# 可迭代的对象，迭代器和生成器
## 1.Sentence第一版：单词序列

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' %(reprlib.repr(self.text))

In [2]:
s = Sentence("'the war has come,' the Warlus said")
s

Sentence("'the war has...e Warlus said")

In [4]:
for word in s:
    print(word)

the
war
has
come
the
Warlus
said


In [5]:
s[1]

'war'

### 序列可迭代的原因 iter 函数

In [8]:
from collections import abc
class Foo:
    def __iter__(self):
        pass

issubclass(Foo, abc.Iterable)

True

In [9]:
f = Foo()
isinstance(f, abc.Iterable)

True

## 可迭代对象与迭代器的对比

In [10]:
s1 = Sentence('pig and pepper')
it1 = iter(s1)
it1

In [11]:
next(it1)

'pig'

In [12]:
next(it1)

'and'

In [13]:
next(it1)

'pepper'

In [14]:
next(it1)

StopIteration: 

In [15]:
list(it1)

[]

## 3.Sentence第二版：典型的迭代器

In [16]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)'%(reprlib.repr(self.text))
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

## 4.Sentence第三版：生成器函数

In [17]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)'%(reprlib.repr(self.text))
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

In [18]:
def gen_123():
    yield 1
    yield 2
    yield 3
for i in gen_123():
    print(i)

1
2
3


In [19]:
g = gen_123()
next(g)

1

In [20]:
next(g)

2

In [21]:
next(g)

3

## 5.Sentence类第四版：惰性实现

In [22]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)'%(reprlib.repr(self.text))
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 6.Sentence 第五版：生成器表达式

In [23]:
def gen_ab():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')

res1 = [x*3 for x in gen_ab()]
res1

start
continue
end


['AAA', 'BBB']

In [24]:
res2 = (x*3 for x in gen_ab())
res2

<generator object <genexpr> at 0x0000028CFF1073C8>

In [25]:
for i in res2:
    print(i)

start
AAA
continue
BBB
end


In [26]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
    
    def __repr__(self):
        return 'Sentence(%s)'%(reprlib.repr(self.text))
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 8.另一个示例：等差数列生成器

In [27]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end = None):
        self.begin = begin
        self.step = step
        self.end = end
    
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        index = 0
        forever = self.end is None
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [28]:
ap = ArithmeticProgression(0,1,7)
list(ap)

[0, 1, 2, 3, 4, 5, 6]

In [30]:
def gen_ari(start, step, end = None):
    result = type(begin + step)(begin)
    index = 0
    forever = end is None
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index
            

In [31]:
import itertools
gen = itertools.count(1, 0.5)
next(gen)

1

In [32]:
next(gen)

1.5

In [33]:
import itertools

def gen_ari(start, step, end):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

## 标准库中的生成函数

In [34]:
def vowel(c):
    return c.lower() in 'aeiou'
list(filter(vowel, 'AsdaEi'))

['A', 'a', 'E', 'i']

In [35]:
import itertools
list(itertools.filterfalse(vowel, 'AsdaEi'))

['s', 'd']

In [36]:
list(itertools.takewhile(vowel, 'AsdaEi'))

['A']

In [37]:
sample = [5, 2, 7, 8, 2, 5, 4, 1, 0]
import itertools
list(itertools.accumulate(sample))

[5, 7, 14, 22, 24, 29, 33, 34, 34]

In [38]:
list(itertools.accumulate(sample, min))

[5, 2, 2, 2, 2, 2, 2, 1, 0]

In [39]:
list(itertools.accumulate(sample, max))

[5, 5, 7, 8, 8, 8, 8, 8, 8]

In [40]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 10, 70, 560, 1120, 5600, 22400, 22400, 0]

In [41]:
list(enumerate('sffgega', 1))

[(1, 's'), (2, 'f'), (3, 'f'), (4, 'g'), (5, 'e'), (6, 'g'), (7, 'a')]

In [42]:
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [43]:
list(itertools.chain('ABC', range(5)))

['A', 'B', 'C', 0, 1, 2, 3, 4]

In [46]:
list(itertools.chain.from_iterable(enumerate('ASDF')))

[0, 'A', 1, 'S', 2, 'D', 3, 'F']

## 10.Python3.3中新出现的句法 yield from

In [47]:
def chain(*iterable):
    for i in iterable:
        yield from i

list(chain('ABC', tuple(range(3))))

['A', 'B', 'C', 0, 1, 2]

## 11.可迭代的归约函数

In [49]:
all([1,2,3])

True

In [50]:
all([0,1,2])

False

In [51]:
any([0,1,2])

True

## 12 深入分析iter函数

In [53]:
from random import randint
def d6():
    return randint(1,6)

d6_iter = iter(d6, 1)
d6_iter

In [54]:
for roll in d6_iter:
    print(roll)

3
6
5
2
4
3
